In [1]:
import sys
sys.path.append("../scripts")
from gen_torch_datasets import gen_dataloaders, to_cuda

In [6]:
train_loader, val_loader, input_shape, num_classes = gen_dataloaders(
    "/home/henrik/Documents/EDTH/CPH2025/drone_acoustics_train_val_data",
    4, 
    144
)

Train samples: 540
Train labels 2160
Val samples: 180
Val labels 720


In [ ]:
import torch.nn as nn
import torch
import torch.optim as optim
from tqdm import tqdm
class SpectrogramCNNExample(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1), nn.BatchNorm2d(16), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1), nn.BatchNorm2d(32), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(), nn.MaxPool2d(2),
        )
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        if x.dim() == 3:
            x = x.unsqueeze(1)
        x = self.features(x)
        x = self.classifier(x)
        return x

model = SpectrogramCNNExample(num_classes=num_classes)
model, device = to_cuda(model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-6)

# Training loop
for epoch in range(100):
    model.train()
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}")
    for X, y in progress_bar:
        optimizer.zero_grad()
        X = X.to(device)
        y = y.to(device)
        outputs = model(X)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

# Validation accuracy
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for X, y in val_loader:
        X = X.to(device)
        y = y.to(device)
        outputs = model(X)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == y).sum().item()
        total += y.size(0)
print(f"Validation accuracy: {correct / total:.2%}")


Number of GPUs: 1
Device 0: NVIDIA GeForce RTX 3050 Ti Laptop GPU


TypeError: 'module' object is not callable